In [1]:
using Distributed
# addprocs(4)

@everywhere begin
    using Revise
    using MendelIHT
    using SnpArrays
    using Random
    using GLM
    using DelimitedFiles
    using Test
    using Distributions
    using LinearAlgebra
    using CSV
    using DataFrames
end

┌ Info: Precompiling MendelIHT [921c7187-1484-5754-b919-5d3ed9ac03c4]
└ @ Base loading.jl:1278


# First simulate multivariate Gaussian traits

In [2]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 20    # number of causal SNPs
r = 2     # number of traits

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray("multivariate_$(r)traits.bed", n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n, 1)
intercepts = [10.0 1.0] # each trait have different intercept

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_Σ, true_b, correct_position = simulate_random_response(xla, k, r, Zu=z*intercepts, overlap=2);

In [3]:
# save true SNP's position and effect size
open("multivariate_$(r)traits_true_beta.txt", "w") do io
    println(io, "snpID,effectsize")
    for pos in correct_position
        println(io, "snp$pos,", true_b[pos])
    end
end

# create `.bim` and `.bam` files using phenotype
make_bim_fam_files(x, Y, "multivariate_$(r)traits")

# Run IHT

In [6]:
Yt = Matrix(Y')
Zt = Matrix(z')
ktrue = k + count(!iszero, intercepts)
@time result = fit_iht(Yt, Transpose(xla), Zt, k=ktrue, verbose=true)

****                   MendelIHT Version 1.3.3                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse Multivariate Gaussian regression
Link functin = IdentityLink()
Sparsity parameter (k) = 22
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Max IHT iterations = 100
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -468.5868178395772, backtracks = 0, tol = 0.10850658256523067
Iteration 2: loglikelihood = 764.0163385489902, backtracks = 0, tol = 0.034074549185494
Iteration 3: loglikelihood = 1153.2401358804402, backtracks = 0, tol = 0.005498036145432363
Iteration 4: loglikelihood = 1179.9110523294462, backtracks = 0, tol = 0.01874759


Compute time (sec):     2.475432872772217
Final loglikelihood:    1274.546945847464
Iterations:             24

Trait 1: IHT estimated 12 nonzero SNP predictors
12×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      200   -1.18575
   2 │      600    1.01649
   3 │     1756   -0.0933257
   4 │     3638   -0.713628
   5 │     4322   -0.310151
   6 │     4678    1.59307
   7 │     5298   -0.615105
   8 │     5797   -1.12359
   9 │     6136    0.794926
  10 │     7091    0.232706
  11 │     7325   -0.376011
  12 │     7360   -0.089475

Trait 1: IHT estimated 1 non-genetic predictors
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1      10.0412

Trait 2: IHT estimated 8 nonzero SNP predictors
8×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │     1278    -0.100992
   2 │     3822     0.561302
   3 │ 

## Check answer

In [7]:
# first beta
β1 = result.beta[1, :]
true_b1_idx = findall(!iszero, true_b[:, 1])
[β1[true_b1_idx] true_b[true_b1_idx, 1]]

11×2 Array{Float64,2}:
 -1.18575   -1.20245
  1.01649    1.00062
 -0.713628  -0.735789
 -0.310151  -0.265599
  1.59307    1.58324
 -0.615105  -0.648652
 -1.12359   -1.14044
  0.794926   0.782584
  0.0       -0.14698
  0.232706   0.231528
 -0.376011  -0.383864

In [8]:
# second beta
β2 = result.beta[2, :]
true_b2_idx = findall(!iszero, true_b[:, 2])
[β2[true_b2_idx] true_b[true_b2_idx, 2]]

9×2 Array{Float64,2}:
  0.0        0.125648
  0.561302   0.602347
 -0.975214  -0.954644
  0.0        0.0161438
  0.47443    0.531549
  1.09704    1.43455
 -1.1012    -1.08864
  0.885201   0.997248
 -1.55465   -1.47439

In [9]:
# non genetic covariates
[result.c intercepts']

2×2 Array{Float64,2}:
 10.0412    10.0
  0.988132   1.0

In [10]:
# covariance matrix
[vec(result.Σ) vec(true_Σ)]

4×2 Array{Float64,2}:
  1.7788    1.79198
 -3.12239  -3.07031
 -3.12239  -3.07031
  7.87963   7.74209

# Test Cross validation

In [35]:
Random.seed!(2020)
Yt = Matrix(Y')
Zt = Matrix(z')
@time mses = cv_iht(Yt, x, Zt);



Crossvalidation Results:
	k	MSE
	1	2888.7160633632498
	2	2624.463471283646
	3	2082.079945131771
	4	1848.6513946661835
	5	1608.392080039595
	6	1290.792877625514
	7	1168.4608330304018
	8	1112.8167471260695
	9	1030.6297699322374
	10	1033.8132619962257
	11	1011.4620380840556
	12	1026.4289617481643
	13	1032.1476493228433
	14	1038.7335045039233
	15	1041.9964100899306
	16	1039.5743394111964
	17	1051.5974076402963
	18	1054.1503547417756
	19	1065.9477448346006
	20	1054.151489969612
272.697336 seconds (108.79 k allocations: 605.203 MiB, 0.04% gc time)


In [36]:
argmin(mses)

11

**Conclusion:** After cross validation, IHT finds 10/12 predictors and 8/10 causal SNPs. 

# GEMMA multivariate results

In [17]:
gemma_df = CSV.read("gemma.result.assoc.txt", DataFrame)

# pvalues
pval_wald = gemma_df[!, :p_wald]
pval_lrt = gemma_df[!, :p_lrt]
pval_score = gemma_df[!, :p_score]

# estimated beta
estim_β1 = gemma_df[!, :beta_1]
estim_β2 = gemma_df[!, :beta_2]

# estimated covariance matrix
estim_σ11 = gemma_df[!, :Vbeta_1_1]
estim_σ12 = gemma_df[!, :Vbeta_1_2]
estim_σ22 = gemma_df[!, :Vbeta_2_2];

In [18]:
correct_snps = [x[1] for x in correct_position]  # truely causal snps
signif_snps = findall(x -> x ≤ 0.05/p, pval_lrt) # gemma's selected snps
signif_snps ∩ correct_snps

12-element Array{Int64,1}:
   46
  901
 1204
 1306
 3160
 4797
 5616
 6072
 6573
 6879
 7236
 9763

**Conclusion:** GEMMA finds 12/20. 

# Compare with univariate IHT

+ `fit_iht`: ~20 times slower
+ `cv_iht`: ~70 times slower

In [49]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
d = Normal
l = canonicallink(d())

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray(undef, n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n)
intercept = 1.0

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_b, correct_position = simulate_random_response(xla, k, d, l, Zu=z*intercept);

In [50]:
@time result = fit_iht(Y, xla, z, d=d(), l=l, k=11)

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse linear regression
Link functin = IdentityLink()
Sparsity parameter (k) = 11
Prior weight scaling = off
Doubly sparse projection = off
Debias = off
Debias = off
Max IHT iterations = 100
Converging when tol < 0.0001:

Iteration 1: loglikelihood = -1577.170794759688, backtracks = 0, tol = 0.609864675283163
Iteration 2: loglikelihood = -1484.8568136206177, backtracks = 0, tol = 0.1269955771967065
Iteration 3: loglikelihood = -1472.9529635904933, backtracks = 0, tol = 0.05823372413927707
Iteration 4: loglikelihood = -1472.5366421393842, backtracks = 1, tol = 0.00450895


IHT estimated 10 nonzero SNP predictors and 1 non-genetic predictors.

Compute time (sec):     0.06683206558227539
Final loglikelihood:    -1472.3905989669602
Iterations:             10

Selected genetic predictors:
10×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      782    -0.437828
   2 │      901     0.747956
   3 │     1204     0.691327
   4 │     1306    -1.42505
   5 │     1655    -0.19456
   6 │     3160    -0.861591
   7 │     3936    -0.147235
   8 │     4201     0.338606
   9 │     4402    -0.126472
  10 │     6879    -1.21895

Selected nongenetic predictors:
1×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        1      1.02016

In [41]:
# compare estimated vs true beta values
[result.beta[correct_position] true_b[correct_position]]

10×2 Array{Float64,2}:
 -0.437828  -0.402269
  0.747956   0.758756
  0.691327   0.729135
 -1.42505   -1.47163
 -0.19456   -0.172668
 -0.861591  -0.847906
  0.338606   0.296183
  0.0       -0.0034339
  0.0        0.125965
 -1.21895   -1.24972

### Cross validation timing

In [92]:
# 1 cores
Random.seed!(2020)
@time cv_iht(Y, x, z);



Crossvalidation Results:
	k	MSE
	1	1221.686764265696
	2	864.1553085156108
	3	661.6430376327284
	4	516.2679723147669
	5	418.2721191658746
	6	342.7931325462592
	7	303.15938053956916
	8	276.6241892508097
	9	278.29180795455045
	10	280.31878745464377
	11	282.10757732881564
	12	285.1294156734012
	13	293.16875008847467
	14	297.43739905389987
	15	296.17300065041906
	16	307.7814278347698
	17	302.0113818893564
	18	312.70262374274137
	19	309.87367962700966
	20	315.6750311115386
  4.315330 seconds (9.35 M allocations: 332.277 MiB, 1.12% gc time)
